### Runpod 설정

In [ ]:
import os

# 모델 저장 경로를 네트워크 볼륨(/workspace) 내부로 지정
os.environ["HF_HOME"] = "/workspace/hf_cache"

In [ ]:
# 필수 라이브러리 설치/업데이트
!pip install transformers accelerate bitsandbytes openai torch nltk huggingface_hub bert-score

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

---

In [85]:
script = "상담사: ▲▲카드 ▲▲▲입니다.\n손님: 예 안녕하세요? 어 저기\n상담사: 네 안녕하십니까?\n손님: ▲▲카드 결 결제하는 그\n손님: 카 저기 결 계좌 이체는 월 변경할려 그러거든요.\n상담사: 결제 계좌를 변경하시겠다 그 말씀이신 거예요?\n손님: 예 예.\n상담사: 알겠습니다. 네 ▲▲▲ 님 본인 맞으십니까?\n손님: 네.\n상담사: 결제 계좌가 저희 쪽에 ▲▲은행 계좌로 등록이 되어 있으신데요. 변경하시고자 하시는 은행명과 계좌 번호 말씀해 주시겠어요, 손님?\n손님: 예.\n손님: ▲▲은행이고요.\n상담사: 네 네.\n손님: 예. ▲▲▲\n상담사: ▲▲▲\n손님: ▲▲▲\n상담사: ▲▲▲\n손님: ▲▲▲\n상담사: ▲▲▲\n손님: ▲▲▲▲\n상담사: ▲▲▲번이요? 말씀해주신 계좌 지금 저희 쪽에서 확인 중에 있습니다. 잠시만 기다려주시겠습니까?\n손님: 예.\n상담사: 네, 손님. 기다려 주셔서 감사합니다. ▲▲카드 이용 대금 출금 이체 계좌 등록에 관한 접수 내용 확인하겠습니다. 생년월일 ▲▲년 ▲▲월 ▲▲일, ▲▲▲ 님 명의로 된 ▲▲은행 ▲▲▲▲▲▲▲▲▲▲▲▲▲ 계좌로\n상담사: 하는 내용이 맞으십니까?\n손님: 네.\n상담사: 확인해주셔서 감사합니다. 요청하신 대로 결제 계좌는 말씀해 주신 ▲▲은행 계좌로 변경 등록 처리해드렸습니다. 그렇기때문에 이 번 ▲▲월 ▲▲일 결제 대금부터는 변경해드린 ▲▲은행 계좌에서 출금 처리가 되는 거고요. 타은행으로 결제\n상담사: 이기 때문에 한도는 결제일 다음 날 원복이 됩니다. 요청하신 대로 결제 계좌는 ▲▲은행 계좌에서 ▲▲은행 계좌로 변경 등록 처리 완료해드렸습니다.\n손님: 네.감사합니다. \n상담사: 감사합니다. 그 외에도 저희 쪽에 장기 카드 대출 등을 이용할 수가 있는데 혹시 카드론 사용 계획은 없으신가요? 유동적이라 변동될 수 있어서요.\n손님: 괜찮습니다.\n상담사: 예 감사합니다. 저는 상담사 ▲▲▲이었습니다."
worktime = 100
emotion_set = ['부정', '긍정', '부정']

In [ ]:
prompt = f"""
상담 스크립트를 평가 기준에 따라 객관적으로 평가하세요

### 제약 사항
1. 모든 감점에는 구체적인 발화 근거를 반드시 제시한다
2. 고객의 감사 표현은 고객 발화에서만 카운트한다
3. 추측, 설명 문장, 자연어 해설 금지 - JSON만 출력한다
4. 총 점수는 60점이며 각 2개 점수의 총합이다

### 상담 스크립트
{script}

### 평가 기준
1. 매뉴얼 준수 (50점에서 감점하는 방식)
intro
- 인사말
  0점: 첫인사 + 마무리 멘트 모두 수행
  -5점: 첫인사 또는 마무리 멘트 누락
- 고객확인
  0점: 고객정보를 고객에게 직접 질문
  -5점: 상담원이 고객정보를 먼저 발화하여 정보 누출

response
- 호응어
  0점: 공감/감성 호응
  -5점: 기운 없음, 짜증 섞인 표현
- 대기 표현
  0점: 대기 표현 모두 수행
  -5점: 대기 표현 누락

explanation
- 커뮤니케이션
  0점: 핵심 요약 + 이해 쉬운 설명
  -5점: 일방적 설명, 단답형
- 알기 쉬운 설명
  0점: 고객 눈높이 설명 + 부연
  -5점: 복잡한 설명/상담자 관점 설명

proactivity
- 적극성
   0점: 적극적 대응
  -5점: 수동적 대응, 대안 없음
- 언어표현
  0점: 정중/경어체/긍정 표현
  -5점: 전문용어, 줄임말, 명령조, 무시 표현

accuracy
- 정확한 업무처리
  0점: 오류 없음
  -10점: 임의 판단으로 업무 오류 발생

2. 고객 감사 표현 (10점)
- 고객 발화 중 감사/칭찬 키워드 포함 시 1회 카운트
- 0회: 0점 / 1회: 5점 / 2회 이상: 10점


### 출력 형식 (JSON)
{{
  "manual_compliance": {{
    "intro": {{
      "score": 0,
      "evidence": []
    }},
    "response": {{
      "score": 0,
      "evidence": []
    }},
    "explanation": {{
      "score": 0,
      "evidence": []
    }},
    "proactivity": {{
      "score": 0,
      "evidence": []
    }},
    "accuracy": {{
      "score": 0,
      "evidence": []
    }},
    "manual_score": "0~50점"
  }},
  "customer_thanks": {{
    "count": 0,
    "thanks_score": "0~10점",
    "evidence": []
  }},
  "final_score": {{
    "total": "manual_score + thanks_score",
    "feedback": ""
  }}
}}
"""

---
### GPT

In [113]:
import time
import json
import openai

In [114]:
def generate_feedback_openai(model_name, prompt):
    start_time = time.perf_counter()
    ttft = 0
    generated_tokens = 0
    content = ""
    first_token_received = False
    
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            response_format={"type": "json_object"},
            stream=True,                              # 토큰 단위로 수신
            stream_options={"include_usage": True}    # 마지막 chunk에 토큰 정보 포함
        )
        
        for chunk in response:
            # 첫번째 토큰이 들어오는 시점 확인 (ttft)
            if not first_token_received and chunk.choices and chunk.choices[0].delta.content:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            # 내용 누적
            if chunk.choices and chunk.choices[0].delta.content:
                content += chunk.choices[0].delta.content
            
            # 토큰 사용량 확인
            if chunk.usage is not None:
                generated_tokens = chunk.usage.completion_tokens
        
        end_time = time.perf_counter()
        total_duration = end_time - start_time
        
        # tps 계산 (생성된 토큰 수 / 전체 소요 시간)
        tps = generated_tokens / total_duration if total_duration > 0 else 0

        result_json = json.loads(content)
        
        metrics = {
                "ttft": round(ttft, 3),
                "tps": round(tps, 2),
                "total_tokens": generated_tokens
            }

        return result_json, metrics
    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}", "오류"

In [95]:
res_gpt_sm, metrics_gpt_sm = generate_feedback_openai("gpt-4.1-mini", prompt)

In [96]:
res_gpt_sm

{'manual_compliance': {'intro': {'score': 0,
   'evidence': ["상담사 첫인사: '▲▲카드 ▲▲▲입니다.'", "상담사 마무리 멘트: '저는 상담사 ▲▲▲이었습니다.'"]},
  'response': {'score': 0,
   'evidence': ["대기 표현: '잠시만 기다려주시겠습니까?', '기다려 주셔서 감사합니다.'",
    "호응어: '네', '알겠습니다', '감사합니다' 등 정중한 표현 사용"]},
  'explanation': {'score': -5,
   'evidence': ["일방적 설명 및 복잡한 설명: '타은행으로 결제 이기 때문에 한도는 결제일 다음 날 원복이 됩니다.' 문장이 복잡하고 고객 눈높이 설명 부족"]},
  'proactivity': {'score': -5,
   'evidence': ['수동적 대응: 고객 요청에만 응대하고 추가 대안 제시 부족',
    "언어표현은 정중하나 '유동적이라 변동될 수 있어서요.' 등 다소 모호한 표현 사용"]},
  'accuracy': {'score': 0, 'evidence': ['업무 처리 정확: 결제 계좌 변경 요청에 대해 정확히 처리함']},
  'manual_score': 40},
 'customer_thanks': {'count': 1,
  'thanks_score': 5,
  'evidence': ["손님: '네.감사합니다.'"]},
 'final_score': {'total': 45,
  'feedback': '상담사는 인사 및 고객확인 절차를 잘 준수하였고, 대기 표현과 호응어도 적절히 사용하였습니다. 다만 설명 부분에서 고객 눈높이 설명이 부족하고 일부 문장이 복잡하여 이해가 어려웠습니다. 또한 적극성 측면에서 추가 대안 제시가 부족하고 일부 표현이 모호한 점이 감점 요인입니다. 업무 처리에는 오류가 없으며, 고객의 감사 표현이 1회 포함되어 점수를 부여하였습니다.'}}

In [97]:
metrics_gpt_sm

{'ttft': 0.567, 'tps': 70.14, 'total_tokens': 522}

---

### sllm

In [ ]:
import re
import json
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

In [ ]:
def generate_summary_sllm(model_name, prompt):
    try:
        # 모델 및 토크나이저 로드
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
        
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        messages = [{"role": "system", "content": prompt}]
        inputs = tokenizer.apply_chat_template(
            messages, 
            tokenize=True, 
            add_generation_prompt=True, 
            return_tensors="pt"
        ).to(model.device)

        # 스트리머 설정
        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        
        # 측정 변수 초기화
        start_time = time.perf_counter()
        ttft = 0
        first_token_received = False
        full_content = ""
        generated_token_count = 0

        # 생성 프로세스를 별도 스레드에서 실행
        generation_kwargs = dict(
            input_ids=inputs,
            streamer=streamer,
            max_new_tokens=512,
            repetition_penalty=1.2,
            do_sample=False,
            temperature=0.1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
        
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

        # 토큰을 하나씩 읽으며 ttft 측정
        for new_text in streamer:
            if not first_token_received and len(new_text) > 0:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            full_content += new_text

        thread.join()
        end_time = time.perf_counter()
        
        total_duration = end_time - start_time
        generated_token_count = len(tokenizer.encode(full_content))
        tps = generated_token_count / total_duration if total_duration > 0 else 0

        # JSON 추출 및 파싱
        try:
            json_match = re.search(r'(\{.*\})', full_content, re.DOTALL)
            if json_match:
                content = json.loads(json_match.group(1))
            else:
                content = {"error": "JSON 형식을 찾을 수 없음", "raw": full_content}
        except json.JSONDecodeError:
            content = {"error": "JSON 파싱 실패", "raw": full_content}

        # 리소스 정리
        del model, tokenizer, inputs
        torch.cuda.empty_cache()

        metrics = {
            "ttft": round(ttft, 3),
            "tps": round(tps, 2),
            "total_tokens": generated_token_count
        }

        return content, metrics

    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}", None

In [115]:
import os
import json
import glob

# 파일 경로 설정
testset_dir = "./testsets"
json_files = glob.glob(os.path.join(testset_dir, "*.json"))

results_list = []

# 파일 루프 시작
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data_list = json.load(f)
        
    for item in data_list:
        # 데이터 추출
        script = item.get('consulting_content', "")
        summary_ref = item['instructions'][0]['data'][0]['output']

        print(f"테스트 실행 중: {item.get('source_id', 'Unknown ID')}")

        # 모델 테스트
        # prompt 생성
        system_prompt = f"""
        상담 스크립트를 평가 기준에 따라 객관적으로 평가하세요

        ### 제약 사항
        1. 모든 감점에는 구체적인 발화 근거를 반드시 제시한다
        2. 고객의 감사 표현은 고객 발화에서만 카운트한다
        3. 추측, 설명 문장, 자연어 해설 금지 - JSON만 출력한다
        4. 총 점수는 60점이며 각 2개 점수의 총합이다

        ### 상담 스크립트
        {script}

        ### 평가 기준
        1. 매뉴얼 준수 (50점에서 감점하는 방식)
        intro
        - 인사말
        0점: 첫인사 + 마무리 멘트 모두 수행
        -5점: 첫인사 또는 마무리 멘트 누락
        - 고객확인
        0점: 고객정보를 고객에게 직접 질문
        -5점: 상담원이 고객정보를 먼저 발화하여 정보 누출

        response
        - 호응어
        0점: 공감/감성 호응
        -5점: 기운 없음, 짜증 섞인 표현
        - 대기 표현
        0점: 대기 표현 모두 수행
        -5점: 대기 표현 누락

        explanation
        - 커뮤니케이션
        0점: 핵심 요약 + 이해 쉬운 설명
        -5점: 일방적 설명, 단답형
        - 알기 쉬운 설명
        0점: 고객 눈높이 설명 + 부연
        -5점: 복잡한 설명/상담자 관점 설명

        proactivity
        - 적극성
        0점: 적극적 대응
        -5점: 수동적 대응, 대안 없음
        - 언어표현
        0점: 정중/경어체/긍정 표현
        -5점: 전문용어, 줄임말, 명령조, 무시 표현

        accuracy
        - 정확한 업무처리
        0점: 오류 없음
        -10점: 임의 판단으로 업무 오류 발생

        2. 고객 감사 표현 (10점)
        - 고객 발화 중 감사/칭찬 키워드 포함 시 1회 카운트
        - 0회: 0점 / 1회: 5점 / 2회 이상: 10점


        ### 출력 형식 (JSON)
        {{
        "manual_compliance": {{
            "intro": {{
            "score": 0,
            "evidence": []
            }},
            "response": {{
            "score": 0,
            "evidence": []
            }},
            "explanation": {{
            "score": 0,
            "evidence": []
            }},
            "proactivity": {{
            "score": 0,
            "evidence": []
            }},
            "accuracy": {{
            "score": 0,
            "evidence": []
            }},
            "manual_score": "0~50점"
        }},
        "customer_thanks": {{
            "count": 0,
            "thanks_score": "0~10점",
            "evidence": []
        }},
        "final_score": {{
            "total": "manual_score + thanks_score",
            "feedback": ""
        }}
        }}
        """
        
        # res, metrics = generate_summary_sllm("kakaocorp/kanana-nano-2.1b-instruct", system_prompt)
        # res, metrics = generate_summary_sllm("kakaocorp/kanana-1.5-8b-instruct-2505", system_prompt)
        # res, metrics = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", system_prompt)
        res, metrics = generate_feedback_openai("gpt-4.1-mini", system_prompt)
            
        # 결과 저장
        results_list.append({
            "id": item.get('source_id'),
            "res": res,
            "metrics": metrics,
        })

테스트 실행 중: 20594
테스트 실행 중: 20596


In [116]:
import os
import pandas as pd

df_new = pd.json_normalize(results_list)
df_new['model'] = "gpt-4.1-mini" 

output_file = "evaluation_results_feedback.csv"

if not os.path.exists(output_file):
    df_new.to_csv(output_file, index=False, mode='w', encoding='utf-8-sig')
else:
    df_new.to_csv(output_file, index=False, mode='a', encoding='utf-8-sig', header=False)

print(f"데이터가 추가되었습니다: {output_file}")

데이터가 추가되었습니다: evaluation_results_feedback.csv


---

In [90]:
def evaluate_call(work_time, emotions):
    work_score = 0
    emotion_score = 0
    
    # 후처리 시간 평가
    time_indicator = 90
    
    if work_time <= time_indicator:
        work_score = 20
    elif work_time >= 90:
        if work_time < 120:
            work_score = 15
        elif work_time < 150:
            work_score = 10
        elif work_time < 200:
            work_score = 5
        else:
            work_score = 0
    
    # 감정 전환 평가
    def get_step_score(before, after):
        if before == after:
            return 3
        
        scores = {
            ("부정", "중립"): 5,
            ("부정", "긍정"): 10,
            ("중립", "긍정"): 10,
            ("중립", "부정"): 0,
            ("긍정", "부정"): 0,
            ("긍정", "중립"): 5
        }
        return scores.get((before, after), 0)

    # 단계별 점수 계산
    score_step_1 = get_step_score(emotions[0], emotions[1]) # 초반 -> 중반
    score_step_2 = get_step_score(emotions[1], emotions[2]) # 중반 -> 후반
    
    emotion_score = score_step_1 + score_step_2
    
    return {
        "work_score": work_score, 
        "emotion_score": emotion_score
        }

In [91]:
score = evaluate_call(worktime, emotion_set)
score

{'work_score': 15, 'emotion_score': 10}